This Python Notebook contains the code used to extract the average beta values for each region of interest specified in the folder 'RAD_data'. It assumes a standard file structure and file type (NIFTI), but the former can be easily altered if necessary. In essence, this program takes 1) a directory of fMRI volumes standardized to the Montreal Neurosciences Institute (MNI) template, 2) a set of MNI coordinates for regions of interest, given in a .csv file, and 3) a .csv file containing Barratt Impulsiveness Scores ('bis_scores.csv') and returns a dataframe in which each row is a single participant and the columns correspond to either input features (from the seeded ROIs) or target variables (BIS scores)

In [3]:
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import plotting, input_data, image
from nilearn.image import mean_img
from sklearn.feature_extraction import image
from sklearn.cluster import FeatureAgglomeration
import matplotlib.pyplot as plt
import numpy as np
import time as time
import pandas as pd

In [2]:
#Set the path to the directory containing the fMRI volumes
path = '/RAD_data/'

In [21]:
class Participant(object):
    """
    Represents a single participant, with an associated preprocessed fMRI and
    BIS scores (including individual responses, subtotals, and cumulative 
    total scores).
    """
    __slots__ = ['participant_id', 'path_to_fMRI_volume', 'bis_total', 'bis_attention_total',
                'bis_coginstable_total','bis_motor_total','bis_preserve_total',
                'bis_selfcontrol_total','bis_cogcomplex_total']
    
    def __init__(self, participant_info, path_to_fMRI_volume):
        self.participant_id = int(participant_info['participant_id'])
        self.path_to_fMRI_volume = path_to_fMRI_volume
        self.bis_total = int(participant_info['bis_total'])
        self.bis_attention_total = int(participant_info['bis_attention_total'])
        self.bis_coginstable_total = int(participant_info['bis_coginstable_total'])
        self.bis_motor_total = int(participant_info['bis_motor_total'])
        self.bis_preserve_total = int(participant_info['bis_preserve_total'])
        self.bis_selfcontrol_total = int(participant_info['bis_selfcontrol_total'])
        self.bis_cogcomplex_total = int(participant_info['bis_cogcomplex_total'])

First, we read in the 'bis_scores.csv' data file and initialize a data frame with the columns from this file. We print out the columns at this point to give an example of what these columns look like

In [16]:
#Read the CSV into an empty dataframe
bis_scores = pd.read_csv(path + 'bis_scores.csv')

#Print out the columns, which are derived from the column names in 'bis_scores.csv'
participant_info = bis_scores.loc[bis_scores['participant_id'] == 243]
print(participant_info)

Empty DataFrame
Columns: [participant_id, bis1, bis2, bis3, bis4, bis5, bis6, bis7, bis8, bis9, bis10, bis11, bis12, bis13, bis14, bis15, bis16, bis17, bis18, bis19, bis20, bis21, bis22, bis23, bis24, bis25, bis26, bis27, bis28, bis29, bis30, bis_total, bis_attention_total, bis_coginstable_total, bis_motor_total, bis_preserve_total, bis_selfcontrol_total, bis_cogcomplex_total]
Index: []

[0 rows x 38 columns]


Next, we read in the Cognitive Circuit Coordinates, given according to the MNI Template in a .csv file, and create columns for these features in our data frame.

In [26]:
cog_circuit_rois = pd.read_csv(path + 'cog_circuit_coords.csv')
cog_circuit_coords = np.vstack((cog_circuit_rois['x'], cog_circuit_rois['y'], cog_circuit_rois['z'])).T
rois = cog_circuit_rois['brain_region']
features = pd.DataFrame(columns=list(bis_scores.columns) + list(rois) + ['path_to_img'])

features

,participant_id,bis1,bis2,bis3,bis4,bis5,bis6,bis7,bis8,bis9,...,bis_selfcontrol_total,bis_cogcomplex_total,DLPFC_left,DLPFC_right,precentral_gyrus_left,precentral_gyrus_right,dACC,dParietal_left,dParietal_right,path_to_img


Additionally, we create a Masker Object that will, using the coordinates from above, find the average value in each fMRI image within a 5mm sphere of the given coordinates and put that value in an appropriately labeled column in the participants dataframe

In [ ]:
masker = input_data.NiftiSpheresMasker(
    seeds = cog_circuit_coords, 
    radius = 5., 
    )

This next cell is the meat of my code. It iterates through each of the participant ID's, checks if that participant ID is associated with an extant fMRI image and a nonempty set of BIS scores, and (if both of those conditions are true), extracts the relevant features

In [27]:
path_start = '/RAD_data/'
path_end = '/000_data_archive/100_fMRI/111_fMRI_stats_spikesonly_FD_fromFile_GO_NO_GO/con_0003_mni.nii'

beta_volumes = []
participants_excluded_for_missing_fMRI = []
participants_excluded_for_missing_bis_scores = []
participants = []
missing_fMRI_volume = True
full_participant_vector = []

num_participants = 0

for i in range(0,412):
    path_to_contrast = path_start + 'RAD_' + str(i) + path_end
    
    #Check to make sure the participant has a contrast in the file system...
    try:
        img = nib.load(path_to_contrast)
        beta_volumes.append(path_to_contrast)
        missing_fMRI_volume = False
    #...otherwise exclude them from consideration
    except FileNotFoundError:
        participants_excluded_for_missing_fMRI.append(i)
        continue
    
    participant_info = bis_scores.loc[bis_scores['participant_id'] == i]
    
    #If the participant has BIS scores recorded in the bis_scores.csv file
    #then include those scores in the patient object, otherwise exclude the
    #patient from consideration
    if len(participant_info.index) == 0 or participant_info.isnull().values.any() or missing_fMRI_volume == True:
        participants_excluded_for_missing_bis_scores.append(i)
    else:
        p = Participant(participant_info, path_to_contrast)
        participants.append(p)
        print('Featurizing participant ' + str(i) + '...')
        fMRI_features = masker.fit_transform(path_to_contrast)    
        full_participant_vector.append(participant_info.values[0].tolist() + fMRI_features[0].tolist() + [path_to_contrast])
        num_participants = num_participants + 1
    
    missing_fMRI_volume = True

    print(str(len(beta_volumes)) + ' participants included')
print(str(len(participants_excluded_for_missing_fMRI)) + 
      ' participants did not have an associated fMRI volume and were excluded')
print(str(len(participants_excluded_for_missing_bis_scores)) + 
      ' participants did not have BIS scores and were excluded')

Featurizing participant 1...
Featurizing participant 2...
Featurizing participant 3...
Featurizing participant 4...
Featurizing participant 5...
Featurizing participant 6...
Featurizing participant 7...
Featurizing participant 8...
Featurizing participant 9...
Featurizing participant 10...
Featurizing participant 11...
Featurizing participant 13...
Featurizing participant 14...
Featurizing participant 15...
Featurizing participant 16...
Featurizing participant 17...
Featurizing participant 18...
Featurizing participant 19...
Featurizing participant 20...
Featurizing participant 22...
Featurizing participant 23...
Featurizing participant 24...
Featurizing participant 25...
Featurizing participant 26...
Featurizing participant 28...
Featurizing participant 29...
Featurizing participant 30...
Featurizing participant 31...
Featurizing participant 32...
Featurizing participant 33...
Featurizing participant 34...
Featurizing participant 35...
Featurizing participant 36...
Featurizing partici

In this final step, we create a dataframe that has each of the feature values and the BIS scores stored for each participant, where the participants are labeled according to participant_id

In [28]:
participant_df = pd.DataFrame(full_participant_vector, columns=list(bis_scores.columns) + list(rois) + ['path_to_img'])
participant_df.to_csv(path + 'participant_df_using_cog_circuit_coords.csv')
participant_df

,participant_id,bis1,bis2,bis3,bis4,bis5,bis6,bis7,bis8,bis9,...,bis_selfcontrol_total,bis_cogcomplex_total,DLPFC_left,DLPFC_right,precentral_gyrus_left,precentral_gyrus_right,dACC,dParietal_left,dParietal_right,path_to_img
0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,2.0,3.0,...,8.0,10.0,0.186089,-0.131558,0.147320,-0.038351,0.102392,-0.060966,-0.187605,C:/Users/Scotty/Documents/PanLab/share/leanew1...
1,2.0,2.0,2.0,1.0,2.0,4.0,2.0,4.0,2.0,4.0,...,15.0,11.0,0.542407,0.806513,0.399728,1.160228,0.900387,0.718495,0.656366,C:/Users/Scotty/Documents/PanLab/share/leanew1...
2,3.0,2.0,2.0,2.0,2.0,2.0,3.0,4.0,1.0,2.0,...,11.0,7.0,0.372603,0.346744,0.452397,0.764894,0.529586,0.677118,0.206760,C:/Users/Scotty/Documents/PanLab/share/leanew1...
3,4.0,3.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,4.0,...,16.0,14.0,0.319340,0.097321,0.211436,-0.269641,-0.151060,0.216431,-0.255214,C:/Users/Scotty/Documents/PanLab/share/leanew1...
4,5.0,2.0,2.0,4.0,4.0,4.0,2.0,1.0,1.0,3.0,...,14.0,13.0,-0.690809,-0.309740,-0.873258,-0.751868,-0.729780,-0.717377,-0.716917,C:/Users/Scotty/Documents/PanLab/share/leanew1...
5,6.0,4.0,1.0,3.0,1.0,2.0,1.0,4.0,2.0,4.0,...,17.0,13.0,0.230076,0.016326,-0.120958,0.272206,0.056923,0.138124,0.041782,C:/Users/Scotty/Documents/PanLab/share/leanew1...
6,7.0,3.0,3.0,1.0,2.0,2.0,3.0,3.0,3.0,3.0,...,17.0,9.0,1.139183,0.536173,0.733851,0.954930,0.741473,1.196933,1.245047,C:/Users/Scotty/Documents/PanLab/share/leanew1...
7,8.0,2.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,2.0,...,12.0,12.0,-0.677713,0.405842,-0.153778,0.433533,1.072146,0.298843,0.539575,C:/Users/Scotty/Documents/PanLab/share/leanew1...
8,9.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,12.0,10.0,0.284126,0.198949,0.125824,-0.001804,-0.117864,0.100775,0.519168,C:/Users/Scotty/Documents/PanLab/share/leanew1...
9,10.0,2.0,3.0,3.0,1.0,3.0,4.0,2.0,3.0,4.0,...,14.0,16.0,0.304522,0.062034,0.351939,0.129803,0.129380,-0.220063,0.404168,C:/Users/Scotty/Documents/PanLab/share/leanew1...
